In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from tqdm import tqdm

from get_data_all_sources import get_data, make_mpstats_request
from source.net_bilstm import BiLSTMAutoencoder # old solution
from source.net_trend_bi import PredictTrendBi # new solution
from source.net_expect import ExpectParams # new exponenta solution
from source.net_seq2seq import Seq2Seq 
from source.exponenta import get_stat

from datetime import datetime, timedelta

import torch
import torch.nn as nn
import requests

DATA_PATH = 'data/'
SOURCE_PATH = 'source/'
PRODUCTS_DIR = 'products/'
WEIGHTS_DIR = 'weights/'
PRODUCTS_PRICES_DIR = 'products_prices/'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def make_tg_report(text) -> None:
    token = '6498069099:AAFtdDZFR-A1h1F-8FvOpt6xIzqjCbdLdsc'
    method = 'sendMessage'
    chat_id = 324956476
    _ = requests.post(
            url='https://api.telegram.org/bot{0}/{1}'.format(token, method),
            data={'chat_id': chat_id, 'text': text} 
        ).json()

In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
model_v1 = BiLSTMAutoencoder(2, 128, 1)
model_v1.load_state_dict(torch.load(WEIGHTS_DIR+'trend_bilstm_cpu', weights_only=False, map_location=device));

model_v2 = PredictTrendBi(3, 128, 2)
model_v2.load_state_dict(torch.load(WEIGHTS_DIR+'trend_2enc_1dec_128_bi', weights_only=False, map_location=device));   

model_v2_exp = ExpectParams(3, 128)
model_v2_exp.load_state_dict(torch.load(WEIGHTS_DIR+'expect_128_1', weights_only=False, map_location=device));  

model_s2s = Seq2Seq(2, 256, 1)
model_s2s.load_state_dict(torch.load(WEIGHTS_DIR+'trend_gru', map_location=device));

In [ ]:
# летний тест с датами до мая
NUM_DAYS = 30
L = os.listdir('products_prices/')
Z = np.zeros((len(L),4+1))
for k,ls in enumerate(tqdm(L)):
    df = pd.read_csv('products_prices/'+ls)
    sku = int(ls[:-4])
    # print(sku)
    # print('================================')
    data = df.iloc[:,1:4].to_numpy(copy=True)
    sum_v1, sum_v2, sum_v2_exp, real = 0,0,0,0 
    for day in range(NUM_DAYS):
        p = data[:len(data)-NUM_DAYS+day+1, 1] # добавляем единицу чтобы завтрашняя цена была в нормировке
        x = (p - p.min())/(p.max()-p.min())
        tomm_price = x[-1]
        x = x[:-1] # убираем обратно
        order = data[:-NUM_DAYS+day, 0]
        balance = data[:-NUM_DAYS+day, 2]
        bal = np.ones_like(balance)
        # bal[balance<order] = 0
        LEN = x.size
        Xv1 = torch.zeros(1,LEN,2,dtype=torch.float32)
        Xv1[:,:,0] = torch.tensor(x)
        Xv1[:,:,1] = torch.tensor(order)
        Xv2 = torch.zeros(1,LEN,3,dtype=torch.float32)
        Xv2[:,:,0] = torch.tensor(bal)
        Xv2[:,:,1] = torch.tensor(x)
        Xv2[:,:,2] = torch.tensor(order)
        
        # old
        out_v1 = model_v1(Xv1).detach().squeeze()
        discount_v1 = order/out_v1.numpy()*out_v1.numpy()[-1]
        b = get_stat(x,discount_v1)
        predict_v1 = np.exp(b[0]+tomm_price*b[1])
        sum_v1 += predict_v1
        
        # new_expect
        out_v2_exp = model_v2_exp(Xv2).detach().squeeze().numpy()
        predict_v2_exp = np.exp(out_v2_exp[1]-tomm_price*out_v2_exp[0])
        sum_v2_exp += predict_v2_exp
        
        # new
        out_v2 = model_v2(Xv2).detach().squeeze()
        discount_v2 = order/out_v2.numpy()*out_v2.numpy()[-1]
        # B = bal == 1
        # x_test = x
        # b = get_stat(x_test[B], discount_v2[B])
        b = get_stat(x, discount_v2)
        predict_v2 = np.exp(b[0]+tomm_price*b[1])
        sum_v2 += predict_v2
        
        real += data[-NUM_DAYS+day, 0]
    Z[k,0] = sum_v1
    Z[k,1] = sum_v2 
    Z[k,2] = sum_v2_exp 
    Z[k,3] = real
    Z[k,4] = sku
    
b1 = Z[:,3] >= 30
Zb = Z[b1,:]
z = np.zeros_like(Zb[:,:3])
z[:,0] = (Zb[:,3] - Zb[:,0])/Zb[:,3]
z[:,1] = (Zb[:,3] - Zb[:,1])/Zb[:,3]
z[:,2] = (Zb[:,3] - Zb[:,2])/Zb[:,3]
z.mean(axis=0), np.abs(z).mean(axis=0), np.sqrt((z**2).mean(axis=0))

In [143]:
b1 = Z[:,3] >= 30
Zb = Z[b1,:]
z = np.zeros_like(Zb[:,[0, 1, 2, 4]])
z[:,0] = (Zb[:,3] - Zb[:,0])/Zb[:,3]
z[:,1] = (Zb[:,3] - Zb[:,1])/Zb[:,3]
z[:,2] = (Zb[:,3] - Zb[:,2])/Zb[:,3]
z[:,3] = Zb[:, 4] 
z.mean(axis=0), np.abs(z).mean(axis=0), np.sqrt((z**2).mean(axis=0))

(array([-3.51991614e-01, -8.38527246e-02, -1.65698066e-02,  8.46262833e+07]),
 array([4.46747368e-01, 2.58370022e-01, 1.99919154e-01, 8.46262833e+07]),
 array([5.27494738e-01, 3.65006518e-01, 2.77106428e-01, 9.39835444e+07]))

In [ ]:
# data = df.iloc[:,1:4].to_numpy(copy=True)
# print(df.columns)
# # bal = np.ones_like(balance)
# order = data[:-NUM_DAYS, 0]
# balance = data[:-NUM_DAYS, 2]
# bal = np.ones_like(balance)
# bal[balance<order] = 0
# bal

In [ ]:
NUM_DAYS = 30
L = os.listdir('new_products/')
Z = np.zeros((len(L),5))
for k,ls in enumerate(tqdm(L)):
    try:
        df = pd.read_csv('new_products/'+ls)
        p = df.price.to_numpy(copy=True)
        x = (p - p.min())/(p.max()-p.min())
        order = df.sales.to_numpy(copy=True)
        balance = df.balance.to_numpy(copy=True)
        bal = np.ones_like(balance)
        # bal[balance<order] = 0
        LEN = p.size
        Xv1 = torch.zeros(1,LEN,2,dtype=torch.float32)
        Xv1[:,:,0] = torch.tensor(x)
        Xv1[:,:,1] = torch.tensor(order)
        Xv2 = torch.zeros(1,LEN,3,dtype=torch.float32)
        Xv2[:,:,0] = torch.tensor(bal)
        Xv2[:,:,1] = torch.tensor(x)
        Xv2[:,:,2] = torch.tensor(order)
        sum_v1, sum_v2, sum_v2_exp, mean_predict, real = 0,0,0,0,0 
        for day in range(NUM_DAYS):
            # old
            out_v1 = model_v1(Xv1[:,:-NUM_DAYS+day,:]).detach().squeeze()
            discount_v1 = order[:-NUM_DAYS+day]/out_v1.numpy()*out_v1.numpy()[-1]
            b = get_stat(x[:-NUM_DAYS+day],discount_v1)
            predict_v1 = np.exp(b[0]+x[LEN-NUM_DAYS+day]*b[1])
            sum_v1 += predict_v1
            
            # new_expect
            out_v2_exp = model_v2_exp(Xv2[:,:-NUM_DAYS+day,:]).detach().squeeze().numpy()
            predict_v2_exp = np.exp(out_v2_exp[1]-x[LEN-NUM_DAYS+day]*out_v2_exp[0])
            sum_v2_exp += predict_v2_exp
            
            # # new
            out_v2 = model_v2(Xv2[:,:-NUM_DAYS+day,:]).detach().squeeze()
            discount_v2 = order[:-NUM_DAYS+day]/out_v2.numpy()*out_v2.numpy()[-1]
            x_test = x[:-NUM_DAYS+day]
            # B = bal[:-NUM_DAYS+day] == 1
            # b = get_stat(x_test[B],discount_v2[B])
            b = get_stat(x_test, discount_v2)
            predict_v2 = np.exp(b[0]+x[LEN-NUM_DAYS+day]*b[1])
            sum_v2 += predict_v2
            
            mean_predict += (predict_v2_exp+predict_v2)/2
            real += order[LEN-NUM_DAYS+day]

        Z[k,0] = sum_v1
        Z[k,1] = sum_v2 
        Z[k,2] = sum_v2_exp 
        Z[k,3] = mean_predict
        Z[k,4] = real
    except:
        print(f'не получилось с {ls}')
make_tg_report('готово')

In [7]:
# с единицами
b1 = Z[:,-1] >= 30
Zb = Z[b1,:]
z = np.zeros_like(Zb[:,:-1])
z[:,0] = (Zb[:,-1] - Zb[:,0])/Zb[:,-1]
z[:,1] = (Zb[:,-1] - Zb[:,1])/Zb[:,-1]
z[:,2] = (Zb[:,-1] - Zb[:,2])/Zb[:,-1]
z[:,3] = (Zb[:,-1] - Zb[:,3])/Zb[:,-1]
z_wnans = z.copy()
z = z[~np.isnan(z).any(axis=1)]
z.mean(axis=0), np.abs(z).mean(axis=0), np.sqrt((z**2).mean(axis=0))

(array([-0.50613507, -0.17565145, -0.20400219, -0.18982682]),
 array([0.6368612 , 0.38128089, 0.36723146, 0.35705982]),
 array([1.0935101 , 0.61001009, 0.60764233, 0.58194687]))

In [12]:
# с нулями
b1 = Z[:,-1] >= 30
Zb = Z[b1,:]
z = np.zeros_like(Zb[:,:-1])
z[:,0] = (Zb[:,-1] - Zb[:,0])/Zb[:,-1]
z[:,1] = (Zb[:,-1] - Zb[:,1])/Zb[:,-1]
z[:,2] = (Zb[:,-1] - Zb[:,2])/Zb[:,-1]
z[:,3] = (Zb[:,-1] - Zb[:,3])/Zb[:,-1]
z_wnans = z.copy()
z = z[~np.isnan(z).any(axis=1)]
z.mean(axis=0), np.abs(z).mean(axis=0), np.sqrt((z**2).mean(axis=0))

(array([-0.50613507, -0.17329789, -0.18412759, -0.17871274]),
 array([0.6368612 , 0.37630269, 0.36147345, 0.35230373]),
 array([1.0935101 , 0.59940399, 0.58905355, 0.56865914]))

In [32]:
b1 = Z[:,-1] <10
Zb = Z[b1,:]
z = np.zeros_like(Zb[:,:-1])
z[:,0] = (Zb[:,-1] - Zb[:,0])/Zb[:,-1]
z[:,1] = (Zb[:,-1] - Zb[:,1])/Zb[:,-1]
z[:,2] = (Zb[:,-1] - Zb[:,2])/Zb[:,-1]
z[:,3] = (Zb[:,-1] - Zb[:,3])/Zb[:,-1]
z_wnans = z.copy()
z = z[~np.isnan(z).any(axis=1)]
z = z[~np.isinf(z).any(axis=1)]
print(f'{z.shape[0]} товаров')
z.mean(axis=0), np.abs(z).mean(axis=0), np.sqrt((z**2).mean(axis=0))

55 товаров


C:\Users\kizee\AppData\Local\Temp\ipykernel_1980\938369932.py:4: RuntimeWarning: divide by zero encountered in divide
  z[:,0] = (Zb[:,-1] - Zb[:,0])/Zb[:,-1]
C:\Users\kizee\AppData\Local\Temp\ipykernel_1980\938369932.py:4: RuntimeWarning: invalid value encountered in divide
  z[:,0] = (Zb[:,-1] - Zb[:,0])/Zb[:,-1]
C:\Users\kizee\AppData\Local\Temp\ipykernel_1980\938369932.py:5: RuntimeWarning: divide by zero encountered in divide
  z[:,1] = (Zb[:,-1] - Zb[:,1])/Zb[:,-1]
C:\Users\kizee\AppData\Local\Temp\ipykernel_1980\938369932.py:5: RuntimeWarning: invalid value encountered in divide
  z[:,1] = (Zb[:,-1] - Zb[:,1])/Zb[:,-1]
C:\Users\kizee\AppData\Local\Temp\ipykernel_1980\938369932.py:6: RuntimeWarning: divide by zero encountered in divide
  z[:,2] = (Zb[:,-1] - Zb[:,2])/Zb[:,-1]
C:\Users\kizee\AppData\Local\Temp\ipykernel_1980\938369932.py:6: RuntimeWarning: invalid value encountered in divide
  z[:,2] = (Zb[:,-1] - Zb[:,2])/Zb[:,-1]
C:\Users\kizee\AppData\Local\Temp\ipykernel_1980\

(array([-12.0033317 , -10.42510797, -10.42890545, -10.42700671]),
 array([12.07419905, 10.53266373, 10.44008809, 10.45306962]),
 array([25.44589614, 29.32204404, 19.47832455, 23.79150876]))

In [35]:
b1 = (Z[:,-1] >=10) & ((Z[:,-1] <20))
Zb = Z[b1,:]
z = np.zeros_like(Zb[:,:-1])
z[:,0] = (Zb[:,-1] - Zb[:,0])/Zb[:,-1]
z[:,1] = (Zb[:,-1] - Zb[:,1])/Zb[:,-1]
z[:,2] = (Zb[:,-1] - Zb[:,2])/Zb[:,-1]
z[:,3] = (Zb[:,-1] - Zb[:,3])/Zb[:,-1]
z_wnans = z.copy()
z = z[~np.isnan(z).any(axis=1)]
print(f'{z.shape[0]} товаров')
z.mean(axis=0), np.abs(z).mean(axis=0), np.sqrt((z**2).mean(axis=0))

36 товаров


(array([-4.63214328, -2.60619376, -2.52408292, -2.56513834]),
 array([4.78154417, 2.7895946 , 2.57413302, 2.65529717]),
 array([6.96479339, 3.99879828, 3.30274657, 3.55278103]))

In [36]:
b1 = (Z[:,-1] >=20) & ((Z[:,-1] <30))
Zb = Z[b1,:]
z = np.zeros_like(Zb[:,:-1])
z[:,0] = (Zb[:,-1] - Zb[:,0])/Zb[:,-1]
z[:,1] = (Zb[:,-1] - Zb[:,1])/Zb[:,-1]
z[:,2] = (Zb[:,-1] - Zb[:,2])/Zb[:,-1]
z[:,3] = (Zb[:,-1] - Zb[:,3])/Zb[:,-1]
z_wnans = z.copy()
z = z[~np.isnan(z).any(axis=1)]
print(f'{z.shape[0]} товаров')
z.mean(axis=0), np.abs(z).mean(axis=0), np.sqrt((z**2).mean(axis=0))

24 товаров


(array([-2.93482489, -1.47172365, -1.71799933, -1.59486149]),
 array([2.98526752, 1.5604779 , 1.79554309, 1.67801049]),
 array([4.05835842, 2.09247748, 2.22970002, 2.10467961]))

In [37]:
b1 = (Z[:,-1] >=30) & ((Z[:,-1] <60))
Zb = Z[b1,:]
z = np.zeros_like(Zb[:,:-1])
z[:,0] = (Zb[:,-1] - Zb[:,0])/Zb[:,-1]
z[:,1] = (Zb[:,-1] - Zb[:,1])/Zb[:,-1]
z[:,2] = (Zb[:,-1] - Zb[:,2])/Zb[:,-1]
z[:,3] = (Zb[:,-1] - Zb[:,3])/Zb[:,-1]
z_wnans = z.copy()
z = z[~np.isnan(z).any(axis=1)]
print(f'{z.shape[0]} товаров')
z.mean(axis=0), np.abs(z).mean(axis=0), np.sqrt((z**2).mean(axis=0))

54 товаров


(array([-1.94077603, -0.98501258, -1.15652197, -1.07076728]),
 array([1.98328143, 1.03712265, 1.1884556 , 1.11084725]),
 array([2.52721102, 1.31417467, 1.38150862, 1.313341  ]))

In [38]:
b1 = (Z[:,-1] >=60) 
Zb = Z[b1,:]
z = np.zeros_like(Zb[:,:-1])
z[:,0] = (Zb[:,-1] - Zb[:,0])/Zb[:,-1]
z[:,1] = (Zb[:,-1] - Zb[:,1])/Zb[:,-1]
z[:,2] = (Zb[:,-1] - Zb[:,2])/Zb[:,-1]
z[:,3] = (Zb[:,-1] - Zb[:,3])/Zb[:,-1]
z_wnans = z.copy()
z = z[~np.isnan(z).any(axis=1)]
print(f'{z.shape[0]} товаров')
z.mean(axis=0), np.abs(z).mean(axis=0), np.sqrt((z**2).mean(axis=0))

375 товаров


(array([-0.29954678, -0.05641097, -0.04410279, -0.05025688]),
 array([0.44297669, 0.28114462, 0.24238803, 0.24307347]),
 array([0.66951912, 0.40289736, 0.3494514 , 0.34865235]))

# Предсказание для 47 (48) товаров с 07 по 21 мая 2024 года

In [4]:
sample = pd.read_excel(DATA_PATH + 'Выборки_товаров_для_ИИ_итоговый_24_05_06.xlsx', sheet_name=3)
skus = sample['Номенклатура'].to_numpy(copy=True)
print(f'кол-во товаров: {np.unique(skus).shape[0]}')
sku = skus[0]
print(sku)

кол-во товаров: 48
17273072


In [ ]:
# date_to = datetime.strptime('06.05.2024', '%d.%m.%Y')
# date_from = (date_to - timedelta(days=366))
# orders_full = pd.read_csv('orders_full.csv', sep=';', parse_dates=['date (date)'], ).iloc[:, :4]
# orders_full.columns = ['date', 'sku', 'sales', 'price']
# np.random.choice(orders_full.sku.unique(), replace=False, size=5000)
# skus = orders_full.sku.sample(250).values
# orders_full.groupby('sku').sales.mean().sort_values(ascending=False)[1000:1250]
# skus = orders_full.groupby('sku').sales.mean().sort_values(ascending=False)[250:750].index
# orders_full.groupby('sku').sales.mean().sort_values(ascending=False)[250:750]
# make_mpstats_request(231046298).json()

In [ ]:
PRODUCTS_DIR = 'new_products/'
for sku in tqdm(skus):
    path_file = PRODUCTS_DIR+f'{sku}.csv'
    if os.path.isfile(path_file):
        df = pd.read_csv(path_file)
    else:
        try:
            df = get_data(sku)
            df.to_csv(path_file, index=False)
        except:
            print(f'не получилось с {sku}')
            continue

In [ ]:
make_tg_report('товары скачались')

In [58]:
path_file = PRODUCTS_DIR+f'{17273072}.csv'
price_path_file = PRODUCTS_PRICES_DIR+f'{17273072}.csv'
df = pd.read_csv(path_file)
df_price = pd.read_csv(price_path_file)
df_two_weeks = df_price[(df_price['date'] > '2024-05-06') & (df_price['date'] < '2024-05-21')].copy()
two_weeks_prices = df_two_weeks.price.to_numpy(copy=True)
two_weeks_orders = df_two_weeks.sales.to_numpy(copy=True)

In [25]:
p = df.price.to_numpy(copy=True)
x = (p - p.min())/(p.max()-p.min()) # потом добавить из тестовых данных цены
order = df.sales.to_numpy(copy=True)
balance = df.balance.to_numpy(copy=True)
bal = np.ones_like(balance)
bal[balance<=order] = 0
LEN = p.size
Xv1 = torch.zeros(1,LEN,2,dtype=torch.float32)
Xv1[:,:,0] = torch.tensor(x)
Xv1[:,:,1] = torch.tensor(order)
Xv2 = torch.zeros(1,LEN,3,dtype=torch.float32)
Xv2[:,:,0] = torch.tensor(bal)
Xv2[:,:,1] = torch.tensor(x)
Xv2[:,:,2] = torch.tensor(order)
sum_v1, sum_v2, sum_v2_exp, real = 0,0,0,0 
# out_v1 = model_v1(Xv1[:,:-NUM_DAYS+day,:]).detach().squeeze()

In [90]:
zeros = torch.zeros(1,NUM_DAYS,3,dtype=torch.float32)
torch.cat((Xv2,zeros),dim=1).shape

torch.Size([1, 384, 3])

In [72]:
model_v1 = BiLSTMAutoencoder(2, 128, 14)
out_v1 = model_v1(Xv1).detach().numpy().squeeze()


NUM_DAYS = 14
discount_v1 = order/out_v1*out_v1[-1]
a, b = get_stat(x, discount_v1)
predict_s2s = model_s2s(Xv1.permute([1, 0, 2]), num_step=NUM_DAYS).detach().numpy().flatten()
predict_v1 = np.exp(a+x[-1]*b)*predict_s2s

In [75]:
predict_v1.mean(), two_weeks_orders.mean()

(0.31274813, 0.14285714285714285)

In [83]:
NUM_DAYS = 14
L = os.listdir('products/')
Z = np.zeros((len(L),4))
Z_using_prices = np.zeros((len(L),1))
for k,ls in enumerate(tqdm(L)):
    df = pd.read_csv('products/'+ls)
    df_price = pd.read_csv('products_prices/'+ls)
    df_two_weeks = df_price[(df_price['date'] > '2024-05-06') & \
        (df_price['date'] < '2024-05-21')].copy()
    two_weeks_prices = df_two_weeks.price.to_numpy(copy=True)
    two_weeks_orders = df_two_weeks.sales.to_numpy(copy=True)
    p = df.price.to_numpy(copy=True)
    x = (p - p.min())/(p.max()-p.min())
    order = df.sales.to_numpy(copy=True)
    balance = df.balance.to_numpy(copy=True)
    bal = np.ones_like(balance)
    # bal[balance==0] = 0
    bal[balance<order] = 0
    LEN = p.size
    Xv1 = torch.zeros(1,LEN,2,dtype=torch.float32)
    Xv1[:,:,0] = torch.tensor(x)
    Xv1[:,:,1] = torch.tensor(order)
    Xv2 = torch.zeros(1,LEN,3,dtype=torch.float32)
    Xv2[:,:,0] = torch.tensor(bal)
    Xv2[:,:,1] = torch.tensor(x)
    Xv2[:,:,2] = torch.tensor(order)
    sum_v1, sum_v2, sum_v2_exp, real = 0,0,0,0 
    
    # TODO: если понадобится, здесь идёт пересчёт цен нормированных 
    all_prices = np.concatenate([p, two_weeks_prices])
    p_min, p_max = all_prices.min(), all_prices.max()
    two_weeks_prices_norm = (two_weeks_prices - p_min)/(p_max - p_min)
    
    # old
    out_v1 = model_v1(Xv1).detach().numpy().squeeze()
    discount_v1 = order/out_v1*out_v1[-1]
    a, b = get_stat(x, discount_v1)
    predict_s2s = model_s2s(Xv1.permute([1,0,2]), num_step=NUM_DAYS).detach().numpy().flatten()
    # берём последнюю известную цену TODO: взять из тестовых данных
    predict_v1 = np.exp(a+two_weeks_prices_norm*b)*predict_s2s
    
    # new_v2
    out_v2 = model_v2(Xv2).detach().numpy().flatten() 
    discount_v2 = order/out_v2*out_v2[-1]
    # B = bal == 1
    # a, b = get_stat(x[B], discount_v2[B])
    a, b = get_stat(x, discount_v2)
    data_perm = Xv2[:, :, 1:]
    data_perm = data_perm.permute([1,0,2])
    predict_s2s = model_s2s(data_perm, num_step=NUM_DAYS).detach().numpy().flatten()
    predict_v2 = np.exp(a+two_weeks_prices_norm*b)*predict_s2s
    
    # expect
    # добавляем маску из нулей на две недели
    # zeros = torch.zeros(1,NUM_DAYS,3,dtype=torch.float32)
    # Xv2 = torch.cat((Xv2,zeros),dim=1)
    
    a, b = model_v2_exp(Xv2).detach().squeeze().numpy()
    predict_v2_exp = np.exp(b-a*two_weeks_prices_norm)
    
    Z[k,0] = predict_v1.sum()
    Z[k,1] = predict_v2.sum() 
    Z[k,2] = predict_v2_exp.sum() 
    Z[k,3] = two_weeks_orders.sum()
    Z_using_prices[k, 0] = ((predict_v2 + predict_v2_exp)/2).sum()

100%|██████████| 48/48 [00:08<00:00,  5.83it/s]


1. balance < order работает лучше (0.69) чем balance <= order (0.92)
2. добавлять маску из нулей на 14 дней работает хуже чем не добавлять
3. брать последнюю цену или цены уже известные из теста
4. как считаются метрики в табличке

In [84]:
z = np.zeros_like(Z[:,:3])
z[:,0] = (Z[:,3] - Z[:,0])/Z[:,3]
z[:,1] = (Z[:,3] - Z[:,1])/Z[:,3]
z[:,2] = (Z[:,3] - Z[:,2])/Z[:,3]
z.mean(axis=0), np.abs(z).mean(axis=0), np.sqrt((z**2).mean(axis=0))

(array([-0.96990881, -0.50021707, -0.51634627]),
 array([1.1159724 , 0.78673643, 0.66869228]),
 array([2.27174745, 1.69286103, 1.04877261]))

In [85]:
z_using_prices = np.zeros_like(Z_using_prices)
z_using_prices[:,0] = (Z[:,3] - Z_using_prices[:,0])/Z[:,3]
z_using_prices.mean(axis=0), np.abs(z_using_prices).mean(axis=0), np.sqrt((z_using_prices**2).mean(axis=0))

(array([-0.50828167]), array([0.70536008]), array([1.29784564]))